# khởi tạo thư viện

In [ ]:
import yaml
import pyodbc
from sqlalchemy import create_engine
import pandas as pd

import sys
sys.path.append(r"G:/My Drive/HỌC TẬP DUE/NNT_năm 4 kì 1/DW & DM/config")
import mdl_db_connection
import mdl_db_query

# connect đến db

In [ ]:
conn_str, engine = mdl_db_connection.connect_db()

# 3.chạy code

lấy ra các cột trong table để phân loại ra các nhóm bảng

In [ ]:
# # Tạo một DataFrame chứa danh sách cột
# columns_list = df_data.columns.to_list()
# columns_df = pd.DataFrame(columns_list, columns=["Column Names"])

# # Đặt tên cho tệp Excel
# excel_filename = "column_names.xlsx"

# # Ghi DataFrame vào tệp Excel
# columns_df.to_excel(excel_filename, index=False)

# print(f"Danh sách cột đã được ghi vào tệp {excel_filename}")
# print(columns_list)

 
## hàm thực hiện truy vấn dữ liệu

### hàm select

In [ ]:
def select_query(query):
    global conn_str  # Khai báo rằng bạn muốn sử dụng biến conn_str toàn cục
    print("CÂU TRUY VẤN")
    print("---")
    print(query)
    print("---")

    # Kết nối đến cơ sở dữ liệu
    conx = pyodbc.connect(conn_str) 
    
    # Đọc dữ liệu từ truy vấn và tạo DataFrame
    df = pd.read_sql(query, conx)
    conx.commit()
    # Đóng kết nối pyodbc
    conx.close()

    # Hiển thị DataFrame
    return df


    # print(df_data.columns.to_list())


### hàm thuc thi

In [ ]:
def execute_query(query, table):
    global conn_str  # Khai báo rằng bạn muốn sử dụng biến conn_str toàn cục
    print("CÂU TRUY VẤN")
    print("---")
    print(query)
    print("---")
    
    try:
        # Kết nối đến cơ sở dữ liệu
        conx = pyodbc.connect(conn_str)

        # Thực hiện truy vấn và tạo bảng
        cursor = conx.cursor()
        cursor.execute(query)

        # Xác nhận thay đổi
        conx.commit()
        print(cursor.rowcount)
        if cursor.rowcount == -1:
            print("Đã tạo bảng thành công")
            print("="*30)
        else:
            print(f"Đã chèn {cursor.rowcount} dòng vào bảng {table}")
            print("="*30)


    except Exception as e:
        print("Đã xảy ra lỗi:\n", e)

    finally:
        # Đóng kết nối pyodbc
        conx.close()

## create table

Khởi tạo thông tin chung cho database

In [ ]:
# Định nghĩa các tên bảng và cột
table_source = 'ETL_SaleDataset'
fact_table = "fact_table"
table_time = "Dim_Time"
table_product = "Dim_Product"
table_region = "Dim_Region"
table_customer = "Dim_Customer"



TẠO BẢNG DIM PRODUCT


In [ ]:

# tạo bảng product
query_create_table_product = f"""
    USE {mdl_db_connection.database}
    DROP TABLE IF EXISTS {table_product};
    CREATE TABLE {table_product} (
        ProductID nvarchar(255),
        Product nvarchar(255),
        Description nvarchar(255),
        Supplier nvarchar(255),
        [Suggested Price] float,
        Price float,
        primary key (ProductID)
    );
"""

#insert dữ lijeuej vào bảng dim_product
query_insert_into_table_product = f"""
    INSERT INTO {table_product} (ProductID, Product, Description, Supplier, [Suggested Price], Price)
    SELECT DISTINCT
        ProductID,
        Product,
        Description,
        Supplier,
        [Suggested Price],
        Price
    FROM {table_source};
"""

# tạo bảng dim product
# execute_query(query_create_table_product,table_product)
mdl_db_query.create_tbl_query(query_create_table_product,table_product)

# insert data vào bảng
mdl_db_query.create_tbl_query(query_insert_into_table_product,table_product)

#### TẠO BẢNG DIM CUSTOMER


In [ ]:
table_source = 'ETL_SaleDataset'
table_customer = 'Dim_Customer'

# tạo bảng product
query_create_table_customer = f"""
    USE {database}
    DROP TABLE IF EXISTS {table_customer};
    CREATE TABLE {table_customer} (
        [CustomerId] int,
        [Fname] NVARCHAR(255),
        [Lname] NVARCHAR(255),
        [Balance] MONEY,
        [Address] NVARCHAR(255),
        [E-mail] NVARCHAR(255),
        [City] NVARCHAR(255),
        [Zip] NVARCHAR(255)
        primary key (CustomerId)
    );
"""

#insert dữ lijeuej vào bảng dim_product
query_insert_into_table_customer = f"""
    INSERT INTO {table_customer} (CustomerId, Fname, Lname, Balance, Address, [E-mail], City, Zip)
    SELECT DISTINCT
        CustomerId,
        Fname,
        Lname,
        Balance,
        Address,
        [E-mail],
        City,
        Zip
    FROM {table_source}
    WHERE CustomerId IS NOT NULL;
"""

# tạo bảng dim customer
execute_query(query_create_table_customer,table_customer)

# insert data vào bảng
execute_query(query_insert_into_table_customer,table_customer)

In [ ]:
# # check
# query = "select * from Dim_Customer"
# select_query(query,'Dim_Customer')


#### TẠO BẢNG DIM REGION

In [ ]:

#tạo bảng region
query_create_table_region = f"""
    USE {database};
    DROP TABLE IF EXISTS {table_region};
    CREATE TABLE {table_region} (
        [RegionID] INT IDENTITY(1,1),
        [Region] NVARCHAR(255),
        primary key (RegionID)
    );
"""

query_insert_into_table_region = f"""
    insert into {table_region} (Region)
    select distinct
        Region
    FROM {table_source}
    where Region is not null
"""


#tạo bảng  dim region
execute_query(query_create_table_region, table_region)

#insert data từ bảng gốc vào
execute_query(query_insert_into_table_region, table_region)



In [ ]:
# query = f'''    
# select distinct
#         Region
# FROM {table_source}
# '''

# query = "select * from Dim_Region"
# select_query(query,'Dim_Region')


#### TẠO BẢNG TIME

In [ ]:

query_create_table_time = f"""
    USE {database}
    DROP TABLE IF EXISTS {table_time};
    CREATE TABLE {table_time} (
        Date_ID INT IDENTITY(1,1) PRIMARY KEY,
        [Datetime] DATETIME,
        [Date] DATE,
        [Year] INT,
        [Quarter] INT,
        [Month] INT,
        [Day] INT,
        [Weekday] INT
    );
"""

# insert dữ liệu vào bảng dim_time

query_insert_into_table_time = f"""
    INSERT INTO {table_time} ([Datetime], [Date], [Year], [Quarter], [Month], [Day], [Weekday])
    SELECT DISTINCT
        [Date] AS [Datetime],
        CONVERT(DATE, [Date], 105) AS [Date],
        YEAR([Date]) AS [Year],
        DATEPART(QUARTER, [Date]) AS [Quarter],
        MONTH([Date]) AS [Month],
        DAY([Date]) AS [Day],
        DATEPART(WEEKDAY, [Date]) AS [Weekday]
    FROM 
        {table_source}
    WHERE [Date] IS NOT NULL;

"""
# tạo bảng dim time
execute_query(query_create_table_time,table_time)

# insert data vào bảng
execute_query(query_insert_into_table_time,table_time)


In [ ]:
# query = "select * from Dim_Time"
# select_query(query,'Dim_Time')

#### TẠO BẢNG FACT SALE

In [ ]:

query_create_fact_table = f"""
  USE {database}
  DROP TABLE IF EXISTS {fact_table};
  CREATE TABLE {fact_table} (
    [RowID] INT IDENTITY(1,1) PRIMARY KEY,
    [OrderID] NVARCHAR(255),
    [Units] DOUBLE PRECISION,
    [Sales] MONEY,
    [COGS] MONEY,
    [Profit] MONEY,

    [CustomerID] INT,
    [ProductID] NVARCHAR(255),
    
    [RegionID] INT,
    [Region] varchar(255),
    
    [DateID] INT,
    [Date] DATETIME,
  );
"""

query_insert_into_fact_table = f"""
  INSERT INTO {fact_table} (
    [OrderID],
    [Units],
    [Sales],
    [COGS],
    [Profit],

    [CustomerID],
    [ProductID],

    [Region],

    [Date]
)
SELECT distinct
    [OrderID],
    [Units],
    [Sales],
    [COGS],
    [Profit],

    [CustomerID],
    [ProductID],

    [Region],

    [Date]
FROM {table_source}
WHERE [OrderID] IS NOT NULL
"""

# tạo bảng dim product
execute_query(query_create_fact_table,fact_table)

# insert data vào bảng
execute_query(query_insert_into_fact_table,fact_table)


tạo id cho bảng fact

In [ ]:

lookup_value_region = f"""
    UPDATE {fact_table}
    SET {fact_table}.RegionID = {table_region}.RegionID
    FROM {fact_table}
    inner JOIN {table_region} ON {fact_table}.[Region] = {table_region}.[Region]


    ALTER TABLE [fact_table]
    DROP COLUMN [Region];

"""


lookup_value_date = f"""
    UPDATE {fact_table}
    SET {fact_table}.DateID = {table_time}.[Date_ID]
    FROM {fact_table}
    inner JOIN {table_time} ON {fact_table}.[Date] = {table_time}.[Datetime]

    ALTER TABLE [fact_table]
    DROP COLUMN [Date]

"""
execute_query(lookup_value_region, fact_table)
execute_query(lookup_value_date, fact_table)

#### data modeling

In [ ]:
# Câu truy vấn để xóa ràng buộc khóa ngoại nếu tồn tại
drop_fk_constraints_query = f"""
IF EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS
           WHERE CONSTRAINT_TYPE = 'FOREIGN KEY'
           AND TABLE_NAME = '{fact_table}')
BEGIN
    ALTER TABLE {fact_table}
    DROP CONSTRAINT FK_fact_date;
    
    ALTER TABLE {fact_table}
    DROP CONSTRAINT FK_fact_product;
    
    ALTER TABLE {fact_table}
    DROP CONSTRAINT FK_fact_region;
    
    ALTER TABLE {fact_table}
    DROP CONSTRAINT FK_fact_customer;
END
"""



# Câu truy vấn để thêm ràng buộc khóa ngoại mới
add_fk_constraints_query = f"""
ALTER TABLE {fact_table}
ADD CONSTRAINT FK_fact_date
FOREIGN KEY(DateID) REFERENCES {table_time}(Date_ID);

ALTER TABLE {fact_table}
ADD CONSTRAINT FK_fact_product
FOREIGN KEY(ProductID) REFERENCES {table_product}(ProductID);

ALTER TABLE {fact_table}
ADD CONSTRAINT FK_fact_region
FOREIGN KEY(RegionID) REFERENCES {table_region}(RegionID);

ALTER TABLE {fact_table}
ADD CONSTRAINT FK_fact_customer
FOREIGN KEY(CustomerID) REFERENCES {table_customer}(CustomerId);
"""




# Kết nối đến cơ sở dữ liệu
conx = pyodbc.connect(conn_str)

# Thực hiện truy vấn và tạo bảng
cursor = conx.cursor()

cursor.execute(drop_fk_constraints_query)
cursor.execute(add_fk_constraints_query)



# Xác nhận thay đổi
conx.commit()
# Đóng kết nối pyodbc
conx.close()


chạy thử
inner join tất cả cac bang

In [ ]:
query = f"""
    select * from fact_table 
    inner join Dim_Product ON fact_table.[ProductID] = Dim_Product.[ProductID]
    inner join Dim_Customer on fact_table.[CustomerID] = Dim_Customer.[CustomerId]
    inner join Dim_Region on fact_table.[RegionID] = Dim_Region.[RegionID]
    inner join Dim_Time on fact_table.[DateID] = Dim_Time.[Date_ID]
"""
df_olap = mdl_db_query.print_query(query)
print(df_olap.head())

In [ ]:
import pyodbc
def get_foreign_key_relationships(conn_str):
    conn_str, engine = mdl_db_connection.connect_db()
    conx = pyodbc.connect(conn_str)
    cursor = conx.cursor()

    # Truy vấn để lấy thông tin về ràng buộc khóa ngoại
    query = """
    SELECT 
        OBJECT_NAME(parent_object_id) AS parent_table,
        col.name AS parent_column,
        OBJECT_NAME(referenced_object_id) AS referenced_table,
        col_ref.name AS referenced_column
    FROM sys.foreign_key_columns fkc
    INNER JOIN sys.columns col
        ON fkc.parent_object_id = col.object_id
        AND fkc.parent_column_id = col.column_id
    INNER JOIN sys.columns col_ref
        ON fkc.referenced_object_id = col_ref.object_id
        AND fkc.referenced_column_id = col_ref.column_id
    WHERE OBJECT_NAME(referenced_object_id) IS NOT NULL
    """
    cursor.execute(query)

    # In ra thông tin mối quan hệ giữa các bảng
    for row in cursor.fetchall():
        parent_table, parent_column, referenced_table, referenced_column = row
        print(f"Table: {parent_table} ({parent_column}) -> Table: {referenced_table} ({referenced_column})")
    conx.close()


# Sử dụng hàm get_foreign_key_relationships
database_name = mdl_db_connection.database
get_foreign_key_relationships(conn_str)


In [ ]:
mdl_db_query.get_foreign_key_relationships()